In [1]:
'''بِسْمِ اللهِ الرَّحْمٰنِ الرَّحِيْمِ'''

'بِسْمِ اللهِ الرَّحْمٰنِ الرَّحِيْمِ'

In [2]:
'''سُبْحَـٰنَكَ لَا عِلْمَ لَنَآ إِلَّا مَا عَلَّمْتَنَآ ۖ إِنَّكَ أَنتَ الْعَلِيمُ الْحَكِيمُ'''

'سُبْحَـٰنَكَ لَا عِلْمَ لَنَآ إِلَّا مَا عَلَّمْتَنَآ ۖ إِنَّكَ أَنتَ الْعَلِيمُ الْحَكِيمُ'

In [3]:
from IPython.core.display import HTML
HTML(r"""
<style>
    * {
        font-family: Monaco;
    }
</style>
""")

In [4]:
import os,sys,time,re,math,gc,warnings
warnings.filterwarnings("ignore")
from IPython.display import clear_output
os.cpu_count()

12

In [5]:
os.environ["POLARS_MAX_THREADS"] = "40"
from pathlib import Path
from tqdm import tqdm
from glob import glob
from concurrent.futures import ThreadPoolExecutor,wait
import numpy as np
import pandas as pd
import polars as pl
import polars.selectors as cs
import plotly.express as px
import plotly.graph_objects as go
import plotly.io as pio
pio.templates["mod"] = go.layout.Template(layout=dict(font=dict(family="Monaco")))
pio.templates.default = "mod+plotly_dark"
from plotly.subplots import make_subplots
from sklearn.metrics import auc,roc_auc_score
%xmode Minimal

Exception reporting mode: Minimal


In [6]:
pl.Config.set_fmt_str_lengths(100)
pl.Config.set_float_precision(2)
pl.Config.set_fmt_float("full")
pl.Config.set_tbl_rows(40)

polars.config.Config

In [7]:
train_previous_application_columns = [
    'education_1138M',
    'rejectreasonclient_4145042M',
    'district_544M',
    'num_group1',
    'rejectreason_755M',
    'postype_4733339M',
    'cancelreason_3545846M'
    ]
train_previous_application_status_columns = [
    'num_group1',
    'num_group2'
    ]
train_total_static_columns = [
    'mobilephncnt_593L',
    'lastrejectreasonclient_4145040M',
    'lastapprcommoditycat_1041M',
    'credamount_770A',
    'numrejects9m_859L',
    'homephncnt_628L',
    'disbursedcredamount_1113A',
    'lastrejectreason_759M',
    'numnotactivated_1143L',
    'sellerplacecnt_915L',
    'numcontrs3months_479L',
    'annuity_780A',
    'clientscnt_533L',
    'applications30d_658L',
    'lastcancelreason_561M',
    'lastrejectcommoditycat_161M',
    'deferredmnthsnum_166L',
    'numactivecredschannel_414L'
    ]
train_static_depth_columns = [
    'education_1103M',
    'education_88M',
    'maritalst_385M',
    'description_5085714M',
    'maritalst_893M'
    ]
train_debit_columns = None
train_deposit_columns = ["amount_416A"]
train_person_1_columns = [
    'education_927M',
    'registaddr_district_1083M',
    'contaddr_district_15M',
    'empladdr_district_926M',
    'empladdr_zipcode_114M',
    'language1_981M'
    ]
train_person_2_columns = ['addres_zip_823M', 'addres_district_368M']
train_other_columns = ['amtdepositbalance_4809441A']
train_tax_registry_1_columns = ['amount_4527230A']
train_tax_registry_2_columns = ['amount_4917619A']
train_tax_registry_3_columns = ['pmtamount_36A']
train_credit_internal_columns = ['contractst_545M',
 'contractst_964M',
 'financialinstitution_382M',
 'classificationofcontr_400M',
 'purposeofcred_874M',
 'description_351M',
 'financialinstitution_591M']

In [8]:
train_base = (
    pl.scan_parquet("Downloads/credit_risk/train/train_base.parquet")
    .drop(["MONTH","WEEK_NUM"])
    .select(
        pl.col("case_id").cast(pl.UInt32),
        pl.col("date_decision").str.to_datetime().dt.month().cast(pl.UInt8).alias("month"),
        pl.col("date_decision").str.to_datetime().dt.week().cast(pl.UInt8).alias("week"),
        pl.col("date_decision").str.to_datetime().dt.year().cast(pl.UInt16).alias("year"),
        pl.col("target").cast(pl.UInt8)
    )
    .drop("date_decision")
    .rename({"case_id":"case_id_base"})
)
train_base.fetch().head()

case_id_base,month,week,year,target
u32,u8,u8,u16,u8
0,1,1,2019,0
1,1,1,2019,0
2,1,1,2019,0
3,1,1,2019,0
4,1,1,2019,1


In [9]:
def rem_train_case_ids(df):
    def get_var_name(df):
        for name,value in globals().items():
            if value is df:
                return name

    return (
        df
        .select(pl.col("case_id").unique())
        .with_context(
            train_base.select(["case_id_base"])
        )
        .select(
            pl.col("case_id_base").is_in(pl.col("case_id")).cast(pl.UInt8).mean().alias(f"% of train_base case_ids in {get_var_name(df)}")
        )
        .collect(streaming=True)
    )

In [10]:
train_previous_application =(
    pl.scan_parquet(Path("Downloads/credit_risk/train/train_applprev_1_*.parquet"))
    .select("case_id",cs.by_name(train_previous_application_columns))
    .select(pl.all().shrink_dtype())
    .with_columns(pl.col("case_id").cast(pl.UInt32),pl.col(pl.String).cast(pl.Categorical))
)
display(train_previous_application.fetch().head())
train_previous_application.pipe(rem_train_case_ids)

case_id,education_1138M,rejectreasonclient_4145042M,district_544M,num_group1,rejectreason_755M,postype_4733339M,cancelreason_3545846M
u32,cat,cat,cat,i8,cat,cat,cat
2,"""P97_36_170""","""a55475b1""","""P136_108_173""",0,"""a55475b1""","""a55475b1""","""a55475b1"""
2,"""P97_36_170""","""a55475b1""","""P136_108_173""",1,"""a55475b1""","""a55475b1""","""a55475b1"""
3,"""P97_36_170""","""a55475b1""","""P131_33_167""",0,"""P94_109_143""","""a55475b1""","""P94_109_143"""
4,"""a55475b1""","""a55475b1""","""P194_82_174""",0,"""a55475b1""","""a55475b1""","""P24_27_36"""
5,"""a55475b1""","""a55475b1""","""P54_133_26""",0,"""a55475b1""","""a55475b1""","""P85_114_140"""


% of train_base case_ids in train_previous_application
f64
0.80


In [11]:
train_previous_application_status = (
    pl.scan_parquet("Downloads/credit_risk/train/train_applprev_2.parquet")
    .select("case_id",cs.by_name(train_previous_application_status_columns))
    .with_columns(pl.all().shrink_dtype())
    .with_columns(pl.col("case_id").cast(pl.UInt32),pl.col(pl.String).cast(pl.Categorical))
)
display(train_previous_application_status.fetch().head())
train_previous_application_status.pipe(rem_train_case_ids)

case_id,num_group1,num_group2
u32,i8,i8
2,1,1
2,0,1
2,0,0
2,1,0
3,0,1


% of train_base case_ids in train_previous_application_status
f64
0.80


In [12]:
train_static_total = (
    pl.scan_parquet("Downloads/credit_risk/train/train_static_0_*.parquet")
    .select("case_id",cs.by_name(train_total_static_columns))
    .with_columns(
        pl.col("case_id").cast(pl.UInt32),
        cs.string().cast(pl.Categorical),
        cs.float().cast(pl.Float32)
    )
    .pipe(lambda df: df.rename({_:" ".join(_.split("_")) for _ in df.columns if _ != 'case_id'}))
)
display(train_static_total.fetch().head())
train_static_total.pipe(rem_train_case_ids)

case_id,mobilephncnt 593L,lastrejectreasonclient 4145040M,lastapprcommoditycat 1041M,credamount 770A,numrejects9m 859L,homephncnt 628L,disbursedcredamount 1113A,lastrejectreason 759M,numnotactivated 1143L,sellerplacecnt 915L,numcontrs3months 479L,annuity 780A,clientscnt 533L,applications30d 658L,lastcancelreason 561M,lastrejectcommoditycat 161M,deferredmnthsnum 166L,numactivecredschannel 414L
u32,f32,cat,cat,f32,f32,f32,f32,cat,f32,f32,f32,f32,f32,f32,cat,cat,f32,f32
0,1.00,"""a55475b1""","""a55475b1""",30000.00,0.00,0.00,30000.00,"""a55475b1""",0.00,0.00,0.00,1917.60,0.00,0.00,"""a55475b1""","""a55475b1""",0.00,0.00
1,1.00,"""a55475b1""","""a55475b1""",19999.80,0.00,0.00,19999.80,"""a55475b1""",0.00,0.00,0.00,3134.00,0.00,0.00,"""a55475b1""","""a55475b1""",0.00,0.00
2,2.00,"""a55475b1""","""a55475b1""",78000.00,0.00,1.00,78000.00,"""a55475b1""",0.00,0.00,0.00,4937.00,0.00,0.00,"""a55475b1""","""a55475b1""",0.00,0.00
3,1.00,"""a55475b1""","""a55475b1""",40000.00,1.00,0.00,40000.00,"""P94_109_143""",0.00,1.00,1.00,4643.60,0.00,1.00,"""P94_109_143""","""a55475b1""",0.00,0.00
4,1.00,"""a55475b1""","""a55475b1""",44000.00,0.00,1.00,44000.00,"""a55475b1""",0.00,0.00,0.00,3390.20,0.00,1.00,"""P24_27_36""","""a55475b1""",0.00,0.00


% of train_base case_ids in train_static_total
f64
1.00


In [13]:
train_static_depth = (
    pl.scan_parquet("Downloads/credit_risk/train/train_static_cb_0.parquet")
    .select("case_id",cs.by_name(train_static_depth_columns))
    .with_columns(
        pl.col("case_id").cast(pl.UInt32),
        cs.string().cast(pl.Categorical),
        cs.float().cast(pl.Float32)
    )
    .pipe(lambda df: df.rename({_:" ".join(_.split("_")) for _ in df.columns if _ != 'case_id'}))
)
display(train_static_depth.fetch().head())
train_static_depth.pipe(rem_train_case_ids)

case_id,education 1103M,education 88M,maritalst 385M,description 5085714M,maritalst 893M
u32,cat,cat,cat,cat,cat
357,"""a55475b1""","""a55475b1""","""a55475b1""","""a55475b1""","""a55475b1"""
381,"""a55475b1""","""a55475b1""","""a55475b1""","""a55475b1""","""a55475b1"""
388,"""a55475b1""","""a55475b1""","""a55475b1""","""a55475b1""","""a55475b1"""
405,"""a55475b1""","""a55475b1""","""a55475b1""","""a55475b1""","""a55475b1"""
409,"""717ddd49""","""a55475b1""","""a7fcb6e5""","""a55475b1""","""a55475b1"""


% of train_base case_ids in train_static_depth
f64
0.98


In [14]:
train_deposit = (
    pl.scan_parquet("Downloads/credit_risk/train/train_deposit_1.parquet")
    .select("case_id",cs.by_name(train_deposit_columns))
    .with_columns(
        pl.col("case_id").cast(pl.UInt32),
        cs.float().cast(pl.Float32)
    )
    .pipe(lambda df: df.rename({_:" ".join(_.split("_")) for _ in df.columns if _ != 'case_id'}))
)
display(train_deposit.fetch().head())
train_deposit.pipe(rem_train_case_ids)

case_id,amount 416A
u32,f32
225,0.00
331,260.37
358,0.00
390,211748.53
390,223.68


% of train_base case_ids in train_deposit
f64
0.07


In [15]:
train_first_person = (
    pl.scan_parquet("Downloads/credit_risk/train/train_person_1.parquet")
    .select(pl.col("case_id").cast(pl.UInt32),cs.by_name(train_person_1_columns),cs.contains("person"))
    .with_columns(cs.string().cast(pl.Categorical),cs.float().cast(pl.UInt8))
    .pipe(lambda df: df.rename({_:" ".join(_.split("_")) for _ in df.columns if not _ == 'case_id'}))
    .fill_null(strategy="forward")
)
display(train_first_person.fetch().head())
train_first_person.pipe(rem_train_case_ids)

case_id,education 927M,registaddr district 1083M,contaddr district 15M,empladdr district 926M,empladdr zipcode 114M,language1 981M,personindex 1023L,persontype 1072L,persontype 792L
u32,cat,cat,cat,cat,cat,cat,u8,u8,u8
0,"""P97_36_170""","""P88_18_84""","""P88_18_84""","""P142_57_166""","""P167_100_165""","""P10_39_147""",0,1,1
0,"""a55475b1""","""a55475b1""","""a55475b1""","""a55475b1""","""a55475b1""","""a55475b1""",1,1,4
0,"""a55475b1""","""a55475b1""","""a55475b1""","""a55475b1""","""a55475b1""","""a55475b1""",2,4,5
0,"""a55475b1""","""a55475b1""","""a55475b1""","""a55475b1""","""a55475b1""","""a55475b1""",2,5,5
1,"""P97_36_170""","""P103_93_94""","""P103_93_94""","""P49_46_174""","""P160_59_140""","""P10_39_147""",0,1,1


% of train_base case_ids in train_first_person
f64
1.00


In [16]:
train_second_person = (
    pl.scan_parquet("Downloads/credit_risk/train/train_person_2.parquet")
    .select(pl.col("case_id").cast(pl.UInt32),cs.by_name(train_person_2_columns).cast(pl.Categorical))
    .pipe(lambda df: df.rename({_:" ".join(_.split("_")) for _ in df.columns if not _ == 'case_id'}))
)
display(train_second_person.fetch().head())
train_second_person.pipe(rem_train_case_ids)

case_id,addres zip 823M,addres district 368M
u32,cat,cat
5,"""a55475b1""","""a55475b1"""
6,"""P10_68_40""","""P55_110_32"""
6,"""P10_68_40""","""P55_110_32"""
6,"""P65_136_169""","""P204_92_178"""
6,"""P10_68_40""","""P191_109_75"""


% of train_base case_ids in train_second_person
f64
0.94


In [17]:
train_first_registry = (
    pl.scan_parquet("Downloads/credit_risk/train/train_tax_registry_a_1.parquet")
    .select(pl.col("case_id").cast(pl.UInt32),cs.by_name(train_tax_registry_1_columns).cast(pl.Float32))
    .pipe(lambda df: df.rename({_:" ".join(_.split("_")) for _ in df.columns if not _ == 'case_id'}))
)
display(train_first_registry.fetch().head())
train_first_registry.pipe(rem_train_case_ids)

case_id,amount 4527230A
u32,f32
28631,1946.00
28631,711.00
28631,3616.40
28631,2600.00
28632,2564.00


% of train_base case_ids in train_first_registry
f64
0.30


In [18]:
train_second_registry = (
    pl.scan_parquet("Downloads/credit_risk/train/train_tax_registry_b_1.parquet")
    .select(pl.col("case_id").cast(pl.UInt32),cs.by_name(train_tax_registry_2_columns).cast(pl.Float32))
    .pipe(lambda df: df.rename({_:" ".join(_.split("_")) for _ in df.columns if not _ == 'case_id'}))
)
display(train_second_registry.fetch().head())
train_second_registry.pipe(rem_train_case_ids)

case_id,amount 4917619A
u32,f32
49435,6885.00
49435,6885.00
49435,6885.00
49435,6885.00
49435,6885.00


% of train_base case_ids in train_second_registry
f64
0.10


In [19]:
train_third_registry = (
    pl.scan_parquet("Downloads/credit_risk/train/train_tax_registry_c_1.parquet")
    .select(pl.col("case_id").cast(pl.UInt32),cs.by_name(train_tax_registry_3_columns).cast(pl.Float32))
    .pipe(lambda df: df.rename({_:" ".join(_.split("_")) for _ in df.columns if not _ == 'case_id'}))
)
display(train_third_registry.fetch().head())
train_third_registry.pipe(rem_train_case_ids)

case_id,pmtamount 36A
u32,f32
357,1100.00
357,1200.00
357,1200.00
357,1200.00
357,628.60


% of train_base case_ids in train_third_registry
f64
0.32


In [20]:
train_credit_internal = (
    pl.scan_parquet("Downloads/credit_risk/train/train_credit_bureau_a_1_*.parquet")
    .select(pl.col('case_id').cast(pl.UInt32),cs.by_name(train_credit_internal_columns).cast(pl.Categorical))
    .pipe(lambda df: df.rename({_:" ".join(_.split("_")) for _ in df.columns if not _ == 'case_id'}))
)
train_credit_internal.fetch().head()

case_id,contractst 545M,contractst 964M,financialinstitution 382M,classificationofcontr 400M,purposeofcred 874M,description 351M,financialinstitution 591M
u32,cat,cat,cat,cat,cat,cat,cat
388,"""7241344e""","""a55475b1""","""a55475b1""","""a55475b1""","""a55475b1""","""a55475b1""","""55b002a9"""
388,"""7241344e""","""a55475b1""","""a55475b1""","""a55475b1""","""a55475b1""","""a55475b1""","""P204_66_73"""
388,"""a55475b1""","""a55475b1""","""a55475b1""","""a55475b1""","""a55475b1""","""a55475b1""","""a55475b1"""
388,"""a55475b1""","""a55475b1""","""a55475b1""","""a55475b1""","""a55475b1""","""a55475b1""","""a55475b1"""
388,"""a55475b1""","""a55475b1""","""a55475b1""","""a55475b1""","""a55475b1""","""a55475b1""","""a55475b1"""


# Test data preprocessing

In [21]:
test_path = Path("Downloads/credit_risk/test")
test_files = sorted(os.listdir(test_path))
test_files_df = pl.DataFrame(
    {
        "index":range(len(test_files)),
        "filename":test_files
    }
)
test_files_df

index,filename
i64,str
0,"""test_applprev_1_0.parquet"""
1,"""test_applprev_1_1.parquet"""
2,"""test_applprev_1_2.parquet"""
3,"""test_applprev_2.parquet"""
4,"""test_base.parquet"""
5,"""test_credit_bureau_a_1_0.parquet"""
6,"""test_credit_bureau_a_1_1.parquet"""
7,"""test_credit_bureau_a_1_2.parquet"""
8,"""test_credit_bureau_a_1_3.parquet"""


In [22]:
test_base = (
    pl.scan_parquet("Downloads/credit_risk/test/test_base.parquet")
    .drop(["MONTH","WEEK_NUM"])
    .select(
        pl.col("case_id").cast(pl.UInt32),
        pl.col("date_decision").str.to_datetime().dt.month().cast(pl.UInt8).alias("month"),
        pl.col("date_decision").str.to_datetime().dt.week().cast(pl.UInt8).alias("week"),
        pl.col("date_decision").str.to_datetime().dt.year().cast(pl.UInt16).alias("year"),
    )
    .drop("date_decision")
    .rename({"case_id":"case_id_base"})
)
test_base.fetch().head()

case_id_base,month,week,year
u32,u8,u8,u16
57543,10,41,2020
57549,10,41,2020
57551,10,41,2020
57552,10,41,2020
57569,10,41,2020


In [23]:
def rem_test_case_ids(df):
    def get_var_name(df):
        for name,value in globals().items():
            if value is df:
                return name

    return (
        df
        .select(pl.col("case_id").unique())
        .with_context(
            test_base.select("case_id_base")
        )
        .select(
            pl.col("case_id_base").is_in(pl.col("case_id")).cast(pl.UInt8).mean().alias(f"% of test_base case_ids in {get_var_name(df)}")
        )
        .collect(streaming=True)
    )

In [24]:
test_previous_application = (
    pl.scan_parquet("Downloads/credit_risk/test/test_applprev_1_*.parquet")
    .select("case_id",cs.by_name(train_previous_application_columns))
    .select(pl.all().shrink_dtype())
    .with_columns(pl.col("case_id").cast(pl.UInt32),pl.col(pl.String).cast(pl.Categorical))
)
display(test_previous_application.fetch().head())
test_previous_application.pipe(rem_test_case_ids)

case_id,education_1138M,rejectreasonclient_4145042M,district_544M,num_group1,rejectreason_755M,postype_4733339M,cancelreason_3545846M
u32,cat,cat,cat,i8,cat,cat,cat
57543,"""a55475b1""","""P94_109_143""","""P197_47_166""",0,"""P99_56_166""","""P149_40_170""","""P94_109_143"""
57549,"""P97_36_170""","""P94_109_143""","""P7_110_89""",0,"""P45_84_106""","""P67_102_161""","""P94_109_143"""
57551,"""a55475b1""","""P94_109_143""","""a55475b1""",0,"""P99_56_166""","""P67_102_161""","""P94_109_143"""
57551,"""P97_36_170""","""P94_109_143""","""P197_47_166""",1,"""P45_84_106""","""P46_145_78""","""P94_109_143"""
57552,"""P97_36_170""","""P94_109_143""","""P11_36_178""",0,"""P45_84_106""","""P46_145_78""","""P94_109_143"""


% of test_base case_ids in test_previous_application
f64
0.60


In [25]:
test_previous_application_status = (
    pl.scan_parquet("Downloads/credit_risk/test/test_applprev_2.parquet")
    .select("case_id",cs.by_name(train_previous_application_status_columns))
    .with_columns(pl.all().shrink_dtype())
    .with_columns(
        pl.col("case_id").cast(pl.UInt32),
        pl.col(pl.String).cast(pl.Categorical)
    )
)
display(test_previous_application_status.fetch().head())
test_previous_application_status.pipe(rem_test_case_ids)

case_id,num_group1,num_group2
u32,i8,i8
57543,0,0
57549,0,2
57549,0,0
57549,0,1
57551,1,1


% of test_base case_ids in test_previous_application_status
f64
0.40


In [26]:
test_static_total = (
    pl.scan_parquet("Downloads/credit_risk/test/test_static_0_*.parquet")
    .select("case_id",cs.by_name(train_total_static_columns))
    .with_columns(
        pl.col("case_id").cast(pl.UInt32),
        cs.string().cast(pl.Categorical),
        cs.float().cast(pl.Float32)
    )
    .pipe(lambda df: df.rename({_:" ".join(_.split("_")) for _ in df.columns if _ != 'case_id'}))
)
display(test_static_total.fetch().head())
test_static_total.pipe(rem_test_case_ids)

case_id,mobilephncnt 593L,lastrejectreasonclient 4145040M,lastapprcommoditycat 1041M,credamount 770A,numrejects9m 859L,homephncnt 628L,disbursedcredamount 1113A,lastrejectreason 759M,numnotactivated 1143L,sellerplacecnt 915L,numcontrs3months 479L,annuity 780A,clientscnt 533L,applications30d 658L,lastcancelreason 561M,lastrejectcommoditycat 161M,deferredmnthsnum 166L,numactivecredschannel 414L
u32,f32,cat,cat,f32,f32,f32,f32,cat,f32,f32,f32,f32,f32,f32,cat,cat,f32,f32
57543,1.00,"""P94_109_143""","""a55475b1""",100000.00,0.00,0.00,100000.00,"""P99_56_166""",0.00,0.00,0.00,7637.20,0.00,0.00,"""P94_109_143""","""P159_130_59""",0.00,0.00
57549,2.00,"""P94_109_143""","""a55475b1""",10000.00,0.00,1.00,10000.00,"""P45_84_106""",0.00,0.00,0.00,902.60,0.00,0.00,"""P94_109_143""","""a55475b1""",0.00,0.00
57551,1.00,"""P94_109_143""","""a55475b1""",40000.00,0.00,0.00,40000.00,"""P99_56_166""",0.00,0.00,0.00,3610.20,0.00,0.00,"""P94_109_143""","""a55475b1""",0.00,0.00
57552,1.00,"""P94_109_143""","""a55475b1""",90000.00,0.00,0.00,90000.00,"""P45_84_106""",0.00,0.00,0.00,6964.40,1.00,0.00,"""P94_109_143""","""a55475b1""",0.00,0.00
57569,1.00,"""a55475b1""","""a55475b1""",50000.00,0.00,0.00,50000.00,"""a55475b1""",0.00,0.00,0.00,5553.40,0.00,0.00,"""a55475b1""","""a55475b1""",0.00,0.00


% of test_base case_ids in test_static_total
f64
1.00


In [27]:
test_static_depth = (
    pl.scan_parquet("Downloads/credit_risk/test/test_static_cb_0.parquet")
    .select(cs.by_name("case_id").cast(pl.UInt32),cs.by_name(train_static_depth_columns).cast(pl.Categorical))
    .pipe(lambda df: df.rename({_:" ".join(_.split("_")) for _ in df.columns if _ != 'case_id'}))
)
display(test_static_depth.fetch().head())
test_static_depth.pipe(rem_test_case_ids)

case_id,education 1103M,education 88M,maritalst 385M,description 5085714M,maritalst 893M
u32,cat,cat,cat,cat,cat
57543,"""a55475b1""","""a55475b1""","""a55475b1""","""2fc785b2""","""a55475b1"""
57549,"""a55475b1""","""a55475b1""","""a55475b1""","""2fc785b2""","""a55475b1"""
57551,"""a55475b1""","""a55475b1""","""a55475b1""","""2fc785b2""","""a55475b1"""
57552,"""a55475b1""","""a55475b1""","""a55475b1""","""2fc785b2""","""a55475b1"""
57569,"""a55475b1""","""a55475b1""","""b6cabe76""","""2fc785b2""","""a55475b1"""


% of test_base case_ids in test_static_depth
f64
1.00


In [28]:
test_deposit = (
    pl.scan_parquet("Downloads/credit_risk/test/test_deposit_1.parquet")
    .select(cs.by_name("case_id").cast(pl.UInt32),cs.float().cast(pl.Float32))
    .pipe(lambda df: df.rename({_:" ".join(_.split("_")) for _ in df.columns if _ != 'case_id'}))
)
display(test_deposit.fetch().head())
test_deposit.pipe(rem_test_case_ids)

case_id,amount 416A
u32,f32
57694,150052.38
57719,368.39
57737,202.00
57774,733.01
57890,1178.49


% of test_base case_ids in test_deposit
f64
0.00


In [29]:
test_first_person = (
    pl.scan_parquet("Downloads/credit_risk/test/test_person_1.parquet")
    .select(
        cs.by_name("case_id").cast(pl.UInt32),
        cs.by_name(train_person_1_columns).cast(pl.Categorical),
        cs.contains("person").cast(pl.UInt32)
    )
    .pipe(lambda df: df.rename({_:" ".join(_.split("_")) for _ in df.columns if not _ == 'case_id'}))
    .fill_null(strategy="forward")
)
display(test_first_person.fetch().head())
test_first_person.pipe(rem_test_case_ids)

case_id,education 927M,registaddr district 1083M,contaddr district 15M,empladdr district 926M,empladdr zipcode 114M,language1 981M,personindex 1023L,persontype 1072L,persontype 792L
u32,cat,cat,cat,cat,cat,cat,u32,u32,u32
57543,"""P97_36_170""","""P107_30_170""","""P107_30_170""","""P16_81_96""","""P144_138_111""","""P209_127_106""",0,1,1
57543,"""a55475b1""","""a55475b1""","""a55475b1""","""a55475b1""","""a55475b1""","""a55475b1""",1,1,5
57543,"""a55475b1""","""a55475b1""","""a55475b1""","""a55475b1""","""a55475b1""","""a55475b1""",1,5,5
57549,"""P97_36_170""","""P7_110_89""","""P7_110_89""","""P7_110_89""","""P166_45_172""","""P10_39_147""",0,1,1
57549,"""a55475b1""","""a55475b1""","""a55475b1""","""a55475b1""","""a55475b1""","""a55475b1""",1,1,5


% of test_base case_ids in test_first_person
f64
0.30


In [30]:
test_second_person = (
    pl.scan_parquet("Downloads/credit_risk/test/test_person_2.parquet")
    .select(cs.by_name("case_id").cast(pl.UInt32),cs.by_name(train_person_2_columns).cast(pl.Categorical))
    .pipe(lambda df: df.rename({_:" ".join(_.split("_")) for _ in df.columns if not _ == 'case_id'}))
)
display(test_second_person.fetch().head())
test_second_person.pipe(rem_test_case_ids)

case_id,addres zip 823M,addres district 368M
u32,cat,cat
57552,"""a55475b1""","""a55475b1"""
57569,"""a55475b1""","""a55475b1"""
57630,"""a55475b1""","""a55475b1"""
57631,"""a55475b1""","""a55475b1"""
57632,"""a55475b1""","""a55475b1"""


% of test_base case_ids in test_second_person
f64
0.70


In [31]:
test_first_registry = (
    pl.scan_parquet("Downloads/credit_risk/test/test_tax_registry_a_1.parquet")
    .select(cs.by_name("case_id").cast(pl.UInt32),cs.by_name(train_tax_registry_1_columns).cast(pl.Float32))
    .pipe(lambda df: df.rename({_:" ".join(_.split("_")) for _ in df.columns if not _ == 'case_id'}))
)
display(test_first_registry.fetch().head())
test_first_registry.pipe(rem_test_case_ids)

case_id,amount 4527230A
u32,f32
57679,1120.00
57679,3640.00
57679,3122.00
57679,2160.00
57679,1494.60


% of test_base case_ids in test_first_registry
f64
0.00


In [32]:
test_second_registry = (
    pl.scan_parquet("Downloads/credit_risk/test/test_tax_registry_b_1.parquet")
    .select(cs.by_name("case_id").cast(pl.UInt32),cs.by_name(train_tax_registry_2_columns).cast(pl.Float32))
    .pipe(lambda df: df.rename({_:" ".join(_.split("_")) for _ in df.columns if not _ == 'case_id'}))
)
display(test_second_registry.fetch().head())
test_second_registry.pipe(rem_test_case_ids)

case_id,amount 4917619A
u32,f32
57543,23976.00
57543,23310.20
57543,25174.80
57543,23976.00
57543,31060.20


% of test_base case_ids in test_second_registry
f64
0.20


In [33]:
test_credit_internal = (
    pl.scan_parquet("Downloads/credit_risk/test/test_credit_bureau_a_1_*.parquet")
    .select(cs.by_name("case_id").cast(pl.UInt32),cs.by_name(train_credit_internal_columns).cast(pl.Categorical))
    .pipe(lambda df: df.rename({_:" ".join(_.split("_")) for _ in df.columns if not _ == 'case_id'}))
)
display(test_credit_internal.fetch().head())
test_credit_internal.pipe(rem_test_case_ids)

case_id,contractst 545M,contractst 964M,financialinstitution 382M,classificationofcontr 400M,purposeofcred 874M,description 351M,financialinstitution 591M
u32,cat,cat,cat,cat,cat,cat,cat
57543,"""b919198c""","""a55475b1""","""a55475b1""","""a55475b1""","""a55475b1""","""a55475b1""","""b619fa46"""
57543,"""a55475b1""","""a55475b1""","""a55475b1""","""a55475b1""","""a55475b1""","""a55475b1""","""a55475b1"""
57543,"""a55475b1""","""a55475b1""","""a55475b1""","""a55475b1""","""a55475b1""","""a55475b1""","""a55475b1"""
57543,"""a55475b1""","""a55475b1""","""a55475b1""","""a55475b1""","""a55475b1""","""a55475b1""","""a55475b1"""
57543,"""a55475b1""","""a55475b1""","""a55475b1""","""a55475b1""","""a55475b1""","""a55475b1""","""a55475b1"""


% of test_base case_ids in test_credit_internal
f64
0.20


In [35]:
_ = {"dataframe":[],"case_ids_present":[]}
__ = {"dataframe":[],"case_ids_present":[]}
for name,val in globals().items():
    if ("train" in name) & ("columns" not in name) & ('rem' not in name) & ("base" not in name):
        _["dataframe"].append(name)
        _["case_ids_present"].append(val.pipe(rem_train_case_ids).item())
    if ("test" in name) & ("rem" not in name) & ("base" not in name) & ("path" not in name) & ("files" not in name):
        __["dataframe"].append(name)
        __["case_ids_present"].append(val.pipe(rem_test_case_ids).item())
_ = pl.DataFrame(_).sort(by="case_ids_present",descending=True)
__ = pl.DataFrame(__).sort(by="case_ids_present",descending=True)
display(_)
__

dataframe,case_ids_present
str,f64
"""train_static_total""",1.00
"""train_first_person""",1.00
"""train_static_depth""",0.98
"""train_second_person""",0.94
"""train_credit_internal""",0.91
"""train_previous_application""",0.80
"""train_previous_application_status""",0.80
"""train_third_registry""",0.32
"""train_first_registry""",0.30


dataframe,case_ids_present
str,f64
"""test_static_total""",1.00
"""test_static_depth""",1.00
"""test_second_person""",0.70
"""test_previous_application""",0.60
"""test_previous_application_status""",0.40
"""test_first_person""",0.30
"""test_second_registry""",0.20
"""test_credit_internal""",0.20
"""test_deposit""",0.00


In [63]:
(
    train_base
    .join(
        train_static_total,
        left_on="case_id_base",
        right_on="case_id",
        how="left"
    )
    .join(
        train_static_depth,
        left_on="case_id_base",
        right_on="case_id",
        how="left"
    )
    .join(
        train_second_person,
        left_on="case_id_base",
        right_on="case_id",
        how='left'
    )
    .collect(streaming=True)
)

case_id_base,month,week,year,target,mobilephncnt 593L,lastrejectreasonclient 4145040M,lastapprcommoditycat 1041M,credamount 770A,numrejects9m 859L,homephncnt 628L,disbursedcredamount 1113A,lastrejectreason 759M,numnotactivated 1143L,sellerplacecnt 915L,numcontrs3months 479L,annuity 780A,clientscnt 533L,applications30d 658L,lastcancelreason 561M,lastrejectcommoditycat 161M,deferredmnthsnum 166L,numactivecredschannel 414L,education 1103M,education 88M,maritalst 385M,description 5085714M,maritalst 893M,addres zip 823M,addres district 368M
u32,u8,u8,u16,u8,f32,cat,cat,f32,f32,f32,f32,cat,f32,f32,f32,f32,f32,f32,cat,cat,f32,f32,cat,cat,cat,cat,cat,cat,cat
0,1,1,2019,0,1.00,"""a55475b1""","""a55475b1""",30000.00,0.00,0.00,30000.00,"""a55475b1""",0.00,0.00,0.00,1917.60,0.00,0.00,"""a55475b1""","""a55475b1""",0.00,0.00,null,null,null,null,null,null,null
1,1,1,2019,0,1.00,"""a55475b1""","""a55475b1""",19999.80,0.00,0.00,19999.80,"""a55475b1""",0.00,0.00,0.00,3134.00,0.00,0.00,"""a55475b1""","""a55475b1""",0.00,0.00,null,null,null,null,null,null,null
2,1,1,2019,0,2.00,"""a55475b1""","""a55475b1""",78000.00,0.00,1.00,78000.00,"""a55475b1""",0.00,0.00,0.00,4937.00,0.00,0.00,"""a55475b1""","""a55475b1""",0.00,0.00,null,null,null,null,null,null,null
3,1,1,2019,0,1.00,"""a55475b1""","""a55475b1""",40000.00,1.00,0.00,40000.00,"""P94_109_143""",0.00,1.00,1.00,4643.60,0.00,1.00,"""P94_109_143""","""a55475b1""",0.00,0.00,null,null,null,null,null,null,null
4,1,1,2019,1,1.00,"""a55475b1""","""a55475b1""",44000.00,0.00,1.00,44000.00,"""a55475b1""",0.00,0.00,0.00,3390.20,0.00,1.00,"""P24_27_36""","""a55475b1""",0.00,0.00,null,null,null,null,null,null,null
